In [1]:

from scipy.sparse import *
import numpy as np
import pickle
import random
import pandas as pd
import nltk


In [2]:
#loades the tweet to embed and the vacabulary from which the embeding matrix was constructed
#data_neg = pd.read_csv('twitter-datasets/train_neg.txt',sep='[\t]', header = None,dtype='str',error_bad_lines = False)
#data_pos = pd.read_csv('twitter-datasets/train_pos.txt',sep='[\t]', header = None,dtype='str',error_bad_lines = False)
sentences_pos=[]
with open('twitter-datasets/train_pos.txt') as pos:
    for line in pos:
        sentences_pos.append(line)
data_pos = pd.DataFrame(sentences_pos)

sentences_neg=[]
with open('twitter-datasets/train_neg.txt') as neg:
    for line in neg:
        sentences_neg.append(line)
data_neg = pd.DataFrame(sentences_neg)

In [3]:
print(data_neg[:10])
len(data_neg)

                                                   0
0  vinco tresorpack 6 ( difficulty 10 of 10 objec...
1  glad i dot have taks tomorrow ! ! #thankful #s...
2  1-3 vs celtics in the regular season = were fu...
3  <user> i could actually kill that girl i'm so ...
4  <user> <user> <user> i find that very hard to ...
5   wish i could be out all night tonight ! <user>\n
6                  <user> i got kicked out the wgm\n
7  rt <user> <user> <user> yes she is ! u tell it...
8                      why is she so perfect <url>\n
9  <user> hi harry ! did u havea good time in aus...


100000

In [4]:
def take_part(data,num_data,seed):
    #input is a pd data frmae
    np.random.seed(seed)
    indexs=np.random.permutation(len(data))
    
    idx=indexs[: num_data]# smaller size
    data_small=data.loc[idx]# takes randomly datas with size 1000 out of the data frame
    data_small.index=[np.array(range(1,num_data+1))-1]# changes the label of the dtatframe to 0,1,2,3,4,5 etc
    return data_small

small_neg=take_part(data_neg,1000,20)
small_pos=take_part(data_pos,1000,20)
print(len(small_neg))
print(len(small_pos))
#small_neg.index=[np.array(range(1,1001))-1]
print(small_neg[:10])
print(small_pos[990:1000])

1000
1000
                                                   0
0  sony dcr-sx 41 flash camcorder w / 60x optical...
1  <user> i wish i could just be happy in one of ...
2  whys everyone talking about chickfila on here ...
3         studying for physics is agonising #exams\n
4  10x24 custom picture frame / poster frame . 75...
5  monkeys ( animals that live in the rain forest...
6  <user> it was the guys who were in after you ....
7                                   <user> it is .\n
8  <user> <user> don't let me start talking about...
9  03x29 custom picture frame / poster frame 2.37...
                                                     0
990  " knock knock " " who's there ? " " rugby " " ...
991  rt <user> <user> for saying that ' i smell ' y...
992  i want a giant aquarium with sea turtles , and...
993                       <user> oh it was amazing x\n
994  <user> can you sign my twitition <url> ? ? i t...
995  " <user> wow i have never had this many birthd...
996  <user> that's all

In [5]:

with open('vocab_cut.txt') as pos:
    for line in pos:
        sentences_pos.append(line)
voca = pd.DataFrame(sentences_pos)

data=pd.concat([small_neg,small_pos],ignore_index=True)
em = np.load('embeddings.npy')

In [6]:
print(em.shape)
print(voca.shape)
print(len(data))
type(data)
data
#if(data[:1000]==small_neg):
#    print('the same')

(101298, 20)
(201298, 1)
2000


,0
0,sony dcr-sx 41 flash camcorder w / 60x optical...
1,<user> i wish i could just be happy in one of ...
2,whys everyone talking about chickfila on here ...
3,studying for physics is agonising #exams\n
4,10x24 custom picture frame / poster frame . 75...
...,...
1995,""" <user> wow i have never had this many birthd..."
1996,<user> that's all that's getting me through to...
1997,"<user> chicken shed is made and ready , i'm go..."
1998,bout to burn it down with <user>\n


In [8]:
tweet_token_list=data[0].apply(lambda x: nltk.word_tokenize(x)) # appplies to the 0 vector a word tokenisation and puts it into a new list

In [9]:
type(tweet_token_list) # is a panda serie not animore a data frame

pandas.core.series.Series

In [11]:
print(tweet_token_list) #problem tokenisation of it's -> [it,'s]

0       [sony, dcr-sx, 41, flash, camcorder, w, /, 60x...
1       [<, user, >, i, wish, i, could, just, be, happ...
2       [whys, everyone, talking, about, chickfila, on...
3       [studying, for, physics, is, agonising, #, exams]
4       [10x24, custom, picture, frame, /, poster, fra...
                              ...                        
1995    [``, <, user, >, wow, i, have, never, had, thi...
1996    [<, user, >, that, 's, all, that, 's, getting,...
1997    [<, user, >, chicken, shed, is, made, and, rea...
1998         [bout, to, burn, it, down, with, <, user, >]
1999    [this, is, <, user, >, your, shit, .., this, c...
Name: 0, Length: 2000, dtype: object


In [52]:
em_dim=em.shape[1]
tweet_matrix=np.zeros(em_dim).reshape(em_dim,-1)

In [53]:
#Atention!! this shit takes time if the data set is to big
for twe in tweet_token_list:
    tweet_vec=np.zeros(em_dim) 
    for token in twe:                #np.where(voca==token)[0] the [0] says that the 1st indice where equality is met was taken used to avoid a list of indices
        ind=np.where(voca==token)[0] #indice of token in the vocablurary and so the position of vector represents this word in the words embedings space 
        if len(ind)==0:              #if token is not in dictionarry word vector is zero
            token_vec=np.zeros(em_dim)#every word is a vector of the embedings vector space
        else:
            token_vec=em[ind,:]#writes finally the corresponding vector of the token 
        tweet_vec=np.vstack((tweet_vec,token_vec))#puts the token vetor in one big tweet vector wich is one feature of the feature matrix
                                                  #tweet_vec.shpae(-1,20)
    if len(tweet_vec)==0:                         #fills the final feature matrix
        tweet_vec=tweet_vec.T.reshape(em_dim,-1)
        tweet_matrix=np.append(tweet_matrix,tweet_vec,axis=1)#tweet_vec.shpae(20,-1)
    else:
        tweet_vec=tweet_vec.T.mean(axis=1).reshape(em_dim,-1)

        tweet_matrix=np.append(tweet_matrix,tweet_vec,axis=1)#apends the mean of the tweet value to get a (20,-1) matrix so summes all the vetors together to get a meaning ful representation of the tweet in the glove vector space
tweet_matrix=np.delete(tweet_matrix,[0,0],axis=1)#delets the first row becaus it is just zero from the initialisation
print(tweet_matrix.shape)
#important! at the beginning are the negative tweets and at the end are the positive tweets
#print(tweet_matrix.shape)
#tweet_matrix

(20, 2001)


TypeError: _save_dispatcher() missing 1 required positional argument: 'arr'

In [60]:
np.save('feature_matrix_1000',tweet_matrix)

In [75]:
tX=np.load('feature_matrix_1000.npy')
print(tX.shape)
print(type(tX))

(20, 2001)
<class 'numpy.ndarray'>


In [76]:
print(tX[:,0])
tX=np.delete(tX,[0,0],axis=1)
print(tX[:,0])
tX.shape

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[-0.08409718 -0.15980797  0.28695514 -0.28721007  0.15514346 -0.07946157
  0.33652397  0.07334604  0.32803222  0.04050116  0.17896864  0.3141617
  0.03127516 -0.30388865  0.20162648 -0.01727124  0.18387582 -0.07168783
  0.00897475 -0.26566464]


(20, 2000)

In [77]:
y_neg=np.zeros(len(small_neg))
y_pos=np.ones(len(small_pos))
print(y_neg.shape)
Y=np.append(y_neg,y_pos)
type(Y)
Y

(1000,)


array([0., 0., 0., ..., 1., 1., 1.])

In [106]:
def split_data(x, y, ratio, seed):
    # set seed
    np.random.seed(seed)
    # split the data based on the given ratio
    index=np.random.permutation(len(y))
    train_size=int(np.floor(ratio*len(y)))
    
    idx_train=index[: train_size]#trainings data indices
    idx_val=index[train_size :]#valdiation data indices
    
    y_train=y[idx_train]#trainings results
    y_val=y[idx_val]#validation results
    
    x_train=x[idx_train,:]#trainings data
    x_val=x[idx_val,:]#validation data
    return y_train, y_val, x_train, x_val
[y_tr,y_te,x_tr,x_te]=split_data(tX.T,Y,0.9,20)

In [107]:
from sklearn.linear_model import SGDClassifier

# Stochastic gradinet decent with hige loss

In [111]:
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=19000,verbose=True)
clf.fit(x_tr, y_tr)
y_pred=clf.predict(x_te)

-- Epoch 1
Norm: 25.96, NNZs: 20, Bias: -1.353863, T: 1800, Avg. loss: 3.519662
Total training time: 0.03 seconds.
-- Epoch 2
Norm: 11.72, NNZs: 20, Bias: 0.677977, T: 3600, Avg. loss: 2.118254
Total training time: 0.03 seconds.
-- Epoch 3
Norm: 12.13, NNZs: 20, Bias: -3.036571, T: 5400, Avg. loss: 1.661402
Total training time: 0.03 seconds.
-- Epoch 4
Norm: 8.98, NNZs: 20, Bias: -0.245252, T: 7200, Avg. loss: 1.450184
Total training time: 0.03 seconds.
-- Epoch 5
Norm: 8.27, NNZs: 20, Bias: -1.389435, T: 9000, Avg. loss: 1.272821
Total training time: 0.03 seconds.
-- Epoch 6
Norm: 8.02, NNZs: 20, Bias: -0.473332, T: 10800, Avg. loss: 1.220993
Total training time: 0.03 seconds.
-- Epoch 7
Norm: 7.02, NNZs: 20, Bias: -1.251260, T: 12600, Avg. loss: 1.140706
Total training time: 0.03 seconds.
-- Epoch 8
Norm: 7.51, NNZs: 20, Bias: 0.732481, T: 14400, Avg. loss: 1.114689
Total training time: 0.03 seconds.
-- Epoch 9
Norm: 7.20, NNZs: 20, Bias: -0.593369, T: 16200, Avg. loss: 1.063303
Tota

In [112]:
acc=np.sum(y_pred==y_te)/len(y_te)
print('accuarcy = '+str(acc))

accuarcy = 0.52


# KNN clasification

In [120]:
from sklearn.neighbors import KNeighborsClassifier

In [125]:
neigh = KNeighborsClassifier(n_neighbors=20)
neigh.fit(x_tr, y_tr)
y_pred=neigh.predict(x_te)

In [126]:
acc=np.sum(y_pred==y_te)/len(y_te)
print('accuarcy = '+str(acc))

accuarcy = 0.56


# SVM 

In [158]:
from sklearn.svm import SVC

In [159]:
SVmod=SVC(degree=1)
SVmod.fit(x_tr, y_tr)
y_pred=SVmod.predict(x_te)

In [160]:
acc=np.sum(y_pred==y_te)/len(y_te)
print('accuarcy = '+str(acc))

accuarcy = 0.575


# Neural Network

In [133]:
from sklearn.neural_network import MLPClassifier

In [147]:
NN = MLPClassifier(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(1000), random_state=1, activation='relu', max_iter=50000)
NN.fit(x_tr, y_tr)
y_pred=NN.predict(x_te)

In [148]:
acc=np.sum(y_pred==y_te)/len(y_te)
print('accuarcy = '+str(acc))

accuarcy = 0.575


In [113]:
arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
arr=np.delete(arr, [0, 0],axis=1)
print(arr.shape)
arr

(3, 3)


array([[ 2,  3,  4],
       [ 6,  7,  8],
       [10, 11, 12]])

In [61]:
a=np.array([1,2,3]).reshape(1,-1)
b=np.array([1,2,3]).reshape(1,-1)
np.stack((a,b),axis=-1)

array([[ 0.        , -0.08409718, -0.0628698 , ..., -0.01176455,
         0.33380607,  0.17915743],
       [ 0.        , -0.15980797, -0.04677629, ..., -0.02081695,
        -0.55361333, -0.15582666],
       [ 0.        ,  0.28695514, -0.44161309, ..., -0.32798236,
         0.54019872, -0.04530718],
       ...,
       [ 0.        , -0.07168783,  0.0118153 , ...,  0.2654626 ,
        -0.38930304,  0.03570304],
       [ 0.        ,  0.00897475, -0.28477148, ..., -0.19180417,
         0.16186536, -0.25459723],
       [ 0.        , -0.26566464,  0.2394526 , ...,  0.22564004,
        -0.12968422,  0.08770686]])

In [68]:
a=np.array([1,2,3])
b=np.array([1,2,3])
a=a.reshape(-1,1)
b=b.reshape(-1,1)
print(a.shape)
np.append(a,b,axis=1).shape

(3, 1)


(3, 2)